In [1]:
import os
import re
import glob
import zipfile
import datetime
import warnings
import traceback

import pyart
import tqdm
import numpy as np
import pandas as pd

import matplotlib.pyplot as pl
import seaborn as sns

import gpmmatch
import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

In [2]:
warnings.simplefilter('ignore')

In [3]:
def remove(flist):
    '''
    Remove file if it exists.
    '''
    flist = [f for f in flist if f is not None]
    for f in flist:
        try:
            os.remove(f)
        except FileNotFoundError:
            pass
    return None

In [30]:
def get_radar_archive_file(date, rid):
    '''
    Return the archive containing the radar file for a given radar ID and a
    given date.
    Parameters:
    ===========
    date: datetime
        Date.
    Returns:
    ========
    file: str
        Radar archive if it exists at the given date.
    '''
    datestr = date.strftime('%Y%m%d')
    file = f"/g/data/rq0/level_1/odim_pvol/{rid}/{date.year}/vol/{rid}_{datestr}.pvol.zip"
    if not os.path.exists(file):
        return None
#         raise FileNotFoundError(f'{file}')

    return file

In [31]:
def extract_zip(inzip, date, path):
    def get_zipfile_name(namelist, date):
        datestr = [re.findall('[0-9]{8}_[0-9]{6}', n)[0] for n in namelist]
        timestamps = np.array([datetime.datetime.strptime(dt, '%Y%m%d_%H%M%S') for dt in datestr], dtype='datetime64')
        pos = np.argmin(np.abs(timestamps - date.to_numpy()))
        delta = np.abs(pd.Timestamp(timestamps[pos]) - date).seconds
        grfile = namelist[pos]

        if delta >= 600:
            raise FileNotFoundError('No file')

        return grfile
    
    with zipfile.ZipFile(inzip) as zid:
        namelist = zid.namelist()
        file = get_zipfile_name(namelist, date)
        zid.extract(file, path=path)
    
    grfile = os.path.join(path, file)
    
    return grfile

In [32]:
def buffer(gpmfile, date, rid):
    inzip = get_radar_archive_file(date, rid)
    if inzip is None:
        return None
    
    try:
        grfile = extract_zip(inzip, date, path)
    except FileNotFoundError:
        print(f'No ground {rid} radar file for {date}.')
        return None
    
    try:
        matchset = gpmmatch.volume_matching(gpmfile,
                                            grfile,    
                                            radar_band='S',
                                            refl_name='reflectivity',
                                            fname_prefix=rid,    
                                            write_output=False)
    except Exception:
        print('!!! ERROR !!!')
        print(gpmfile)
        print(date)
        traceback.print_exc()
        return None
    
    remove([grfile])
    
    return matchset

In [33]:
def savedata(datafile, rid):    
    dtime = pd.Timestamp(datafile.gpm_overpass_time)    
    datestr = dtime.strftime('%Y%m%d')
    timestr = dtime.strftime('%H%M%S')
    
    outpath = os.path.join(".", rid)
    try:
        os.mkdir(outpath)
    except FileExistsError:
        pass
    
    outfile = os.path.join(outpath, f"vmatch.{rid}.{datestr}.{timestr}.nc"   )
    if not os.path.exists(outfile):
        datafile.to_netcdf(outfile)
    
    return None

In [34]:
config_files = sorted(glob.glob('/scratch/kl02/vhl548/gpm_output/overpass/*.csv'))
path = '/scratch/kl02/vhl548/unzipdir'

In [35]:
for config in tqdm.tqdm_notebook(config_files, total=len(config_files)):
    rid = os.path.basename(config)[-6:-4]
    df = pd.read_csv(config, parse_dates=['date'], header=None, names=['date', 'name', 'lon', 'lat', 'nprof', 'source'])

    argslist = []
    for n in range(len(df)):
        g = df.source[n]
        d = df.date[n]
        argslist.append((g, d, rid))

    bag = db.from_sequence(argslist).starmap(buffer)
    with ProgressBar():
        rslt = bag.compute()

    rslt = [r for r in rslt if r is not None]

    for r in tqdm.tqdm_notebook(rslt, total=len(rslt)):
        savedata(r, rid)
        
    break

[##                                      ] | 7% Completed |  0.1sNo ground 01 radar file for 2017-07-25 02:39:22.
No ground 01 radar file for 2017-07-25 02:39:22.
[######                                  ] | 16% Completed |  0.5s!!! ERROR !!!
/g/data/rq0/admin/calibration/sr_data/gpm_data/2017/08/28/2A-CS-Tasmania.GPM.Ku.V7-20170308.20170828-S163745-E163852.019883.V05A.HDF5
2017-08-28 16:38:52


Traceback (most recent call last):
  File "<ipython-input-32-ca61466cbf69>", line 13, in buffer
    matchset = gpmmatch.volume_matching(gpmfile,
  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/gpmmatch.py", line 73, in volume_matching
    gpmset, radar = data_load_and_checks(gpmfile,
  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/io.py", line 399, in data_load_and_checks
    raise BrightBandError('No BB')
gpmmatch.io.BrightBandError: No BB


[######                                  ] | 16% Completed |  0.6s!!! ERROR !!!
/g/data/rq0/admin/calibration/sr_data/gpm_data/2017/09/04/2A-CS-Tasmania.GPM.Ku.V7-20170308.20170904-S053743-E053951.019984.V05A.HDF5
2017-09-04 05:37:43


Traceback (most recent call last):


[######                                  ] | 16% Completed |  0.7s

...

[#####################################   ] | 92% Completed |  3min 56.8s

  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/gpmmatch.py", line 73, in volume_matching
    gpmset, radar = data_load_and_checks(gpmfile,
  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/io.py", line 399, in data_load_and_checks
    raise BrightBandError('No BB')
gpmmatch.io.BrightBandError: No BB


[######################################  ] | 96% Completed |  4min 24.4s!!! ERROR !!!
/g/data/rq0/admin/calibration/sr_data/gpm_data/2019/10/19/2A-CS-AUS-East.GPM.Ku.V8-20180723.20191019-S040151-E040757.032039.V06A.HDF5
2019-10-19 04:02:36


Traceback (most recent call last):
  File "<ipython-input-32-ca61466cbf69>", line 13, in buffer
    matchset = gpmmatch.volume_matching(gpmfile,
  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/gpmmatch.py", line 73, in volume_matching
    gpmset, radar = data_load_and_checks(gpmfile,
  File "/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/gpmmatch/io.py", line 399, in data_load_and_checks
    raise BrightBandError('No BB')
gpmmatch.io.BrightBandError: No BB


[########################################] | 100% Completed |  6min 52.5s


In [14]:
rid = os.path.basename(config)[-6:-4]
df = pd.read_csv(config, parse_dates=['date'], header=None, names=['date', 'name', 'lon', 'lat', 'nprof', 'source'])

argslist = []
for n in range(len(df)):
    g = df.source[n]
    d = df.date[n]
    argslist.append((g, d, rid))

In [15]:
rid

'01'

In [17]:
bag = db.from_sequence(argslist).starmap(buffer)
with ProgressBar():
    rslt = bag.compute()

[########################################] | 100% Completed |  0.5s
